In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image

from config import CONSOLIDATED_RESULTS_DIR, FIG_RESULTS_DIR
from drawutil import utcolors, mpl_config

In [ ]:
# global setting
SCALE = 1 # scale = 1.0 for write-up dimensions
mpl_config(SCALE)

In [ ]:
casestudies = [
    'imelda',
    'harvey',
]

approaches = [
    'stochastic',
    'robust',
]

approach_names = {
    'stochastic': 'SP',
    'robust': 'RO',
}

pftypes = {
    'imelda': [
        'dc',
        'lpacc',
        'lpacf',
        'qpac',
    ],
    'harvey': [
        'dc',
        'lpacc',
    ]
}

pftype_names = {
    'dc': 'DC',
    'lpacc': 'LPAC-C',
    'lpacf': 'LPAC-F',
    'qpac': 'QPAC',
}

r_hat = 3

In [ ]:
df = pd.read_csv(os.path.join(CONSOLIDATED_RESULTS_DIR, f'times-r{r_hat}.csv'), header=None)
df.columns = ['f', 'pftype', 'casestudy', 'approach', 'time']
df.set_index(['f', 'pftype', 'casestudy', 'approach'], inplace=True)
df['time'] = df['time'].str.strip()

def time_convert(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    return (hours * 60 + minutes) * 60 + seconds

plt.show()
df['seconds'] = df['time'].apply(time_convert)

In [ ]:
# plot sizing overhead
plot_num_w = 2
plot_num_h = 1
lo_pad_w, md_pad_w, hi_pad_w, plot_dim_w = 0.625 * SCALE, 0.125 * SCALE, 0.250 * SCALE, 2.750 * SCALE
lo_pad_h, md_pad_h, hi_pad_h, plot_dim_h = 0.500 * SCALE, 0.500 * SCALE, 0.250 * SCALE, 1.750 * SCALE
fig_w = lo_pad_w + hi_pad_w + md_pad_w * (plot_num_w - 1) + plot_dim_w * plot_num_w
fig_h = lo_pad_h + hi_pad_h + md_pad_h * (plot_num_h - 1) + plot_dim_h * plot_num_h
fig, axes = plt.subplots(plot_num_h, plot_num_w,
                         figsize=(fig_w, fig_h),
                         sharey=True)
plt.subplots_adjust(left=lo_pad_w / fig_w,
                    bottom=lo_pad_h / fig_h,
                    right=(fig_w - hi_pad_w) / fig_w,
                    top=(fig_h - hi_pad_h) / fig_h,
                    wspace=md_pad_w / plot_dim_w,
                    hspace=md_pad_h / plot_dim_h)

for col, casestudy in enumerate(casestudies):
    for idxp, pftype in enumerate(pftypes[casestudy]):
        for idxa, approach in enumerate(approaches):
            mask = (df.index.get_level_values('pftype') == pftype) &\
                   (df.index.get_level_values('casestudy') == casestudy) &\
                   (df.index.get_level_values('approach') == approach)
            sr = df.loc[mask, 'seconds']
            sr.index = sr.index.get_level_values('f')
            print(casestudy, approach, pftype, sr.idxmin(), sr.min(), sr.idxmax(), sr.max())
            sr.plot(ax=axes[col],
                    label=f'({pftype_names[pftype]}, {approach_names[approach]})', 
                    color=utcolors[idxa + idxp * len(approaches)])
    axes[col].set_xlabel('$f$')

axes[0].set_ylabel('Time to Optimal Solution (s)')
axes[0].legend()
axes[1].legend()
axes[0].set_ylim([1e1, 1e5])
axes[0].set_yscale('log')
axes[0].set_title('Imelda')
axes[1].set_title('Harvey')

axes[0].legend(loc='upper center', ncol=2)
axes[1].legend(loc='lower center', ncol=2)

image_jpg = os.path.join(FIG_RESULTS_DIR, f'ijoc-solution-times.jpg')
image_eps = os.path.join(FIG_RESULTS_DIR, f'ijoc-solution-times.eps')
plt.savefig(image_jpg, format='jpg', dpi=256)
plt.savefig(image_eps, format='eps')
plt.close()
display(Image(image_jpg))